<a href="https://colab.research.google.com/github/hidhartha/Movie_recommender_and_Sentiment_analysis/blob/main/Movie_Recommender_System_and_Sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Movie Recommendation System and Sentiment analysis of movie reviews.
Datasets used:

*   IMDB Top 1000 movie dataset from Kaggle for movie recommendation.
*   IMDB Dataset of 50k movie reviews from Kaggle for sentiment analysis. 


In [1]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1. Movie Recommendation system

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Movie_Recommender_system_ and_sentiment_analysis/imdb_top_1000 (1).csv')

In [4]:
df.head(2)

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"


In [5]:
df.shape

(1000, 16)

**Checking for duplicate rows in the dataset.**

In [6]:
df.duplicated().sum()

0

**There are no duplicate rows in the dataset.**

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Poster_Link    1000 non-null   object 
 1   Series_Title   1000 non-null   object 
 2   Released_Year  1000 non-null   object 
 3   Certificate    899 non-null    object 
 4   Runtime        1000 non-null   object 
 5   Genre          1000 non-null   object 
 6   IMDB_Rating    1000 non-null   float64
 7   Overview       1000 non-null   object 
 8   Meta_score     843 non-null    float64
 9   Director       1000 non-null   object 
 10  Star1          1000 non-null   object 
 11  Star2          1000 non-null   object 
 12  Star3          1000 non-null   object 
 13  Star4          1000 non-null   object 
 14  No_of_Votes    1000 non-null   int64  
 15  Gross          831 non-null    object 
dtypes: float64(2), int64(1), object(13)
memory usage: 125.1+ KB


In [8]:
df.isnull().sum()

Poster_Link        0
Series_Title       0
Released_Year      0
Certificate      101
Runtime            0
Genre              0
IMDB_Rating        0
Overview           0
Meta_score       157
Director           0
Star1              0
Star2              0
Star3              0
Star4              0
No_of_Votes        0
Gross            169
dtype: int64

**Selection the necessary columns which contains textual data**

In [9]:
df1 = df[['Series_Title' ,'Genre', 'Overview','Director','Star1', 'Star2','Star3', 'Star4' ]]

In [10]:
df1.head(1)

,Series_Title,Genre,Overview,Director,Star1,Star2,Star3,Star4
0,The Shawshank Redemption,Drama,Two imprisoned men bond over a number of years...,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler


In [12]:
def convert(string):
    li = list(string.split(" "))
    return li

In [13]:
df1['Star1'] = df1['Star1'].apply(convert)

In [14]:
df1['Star1']

0            [Tim, Robbins]
1          [Marlon, Brando]
2         [Christian, Bale]
3              [Al, Pacino]
4            [Henry, Fonda]
               ...         
995       [Audrey, Hepburn]
996     [Elizabeth, Taylor]
997       [Burt, Lancaster]
998    [Tallulah, Bankhead]
999         [Robert, Donat]
Name: Star1, Length: 1000, dtype: object

In [15]:
df1['Genre'] = df1['Genre'].apply(convert)
df1['Overview'] = df1['Overview'].apply(convert)
df1['Star2'] = df1['Star2'].apply(convert)
df1['Star3'] = df1['Star3'].apply(convert)
df1['Star4'] = df1['Star4'].apply(convert)
df1['Director'] = df1['Director'].apply(convert)

In [16]:
df1.head(1)

,Series_Title,Genre,Overview,Director,Star1,Star2,Star3,Star4
0,The Shawshank Redemption,[Drama],"[Two, imprisoned, men, bond, over, a, number, ...","[Frank, Darabont]","[Tim, Robbins]","[Morgan, Freeman]","[Bob, Gunton]","[William, Sadler]"


In [17]:
df1['Director'] =df1['Director'].apply(lambda x:[i.replace(" ","") for i in x])

Combining all the texts in the columns(Genre ,Overview ,Star1 ,Star2 ,Star3 ,Star4) into a single column tags.

In [18]:
df1['tags'] = df1['Genre'] + df1['Overview'] + df1['Director'] + df1['Star1'] + df1['Star2'] + df1['Star3'] + df1['Star4']

In [19]:
df1.drop(['Genre', 'Overview', 'Director', 'Star1', 'Star2', 'Star3', 'Star4'], axis=1, inplace=True)

In [20]:
df1.head(5)

,Series_Title,tags
0,The Shawshank Redemption,"[Drama, Two, imprisoned, men, bond, over, a, n..."
1,The Godfather,"[Crime,, Drama, An, organized, crime, dynasty'..."
2,The Dark Knight,"[Action,, Crime,, Drama, When, the, menace, kn..."
3,The Godfather: Part II,"[Crime,, Drama, The, early, life, and, career,..."
4,12 Angry Men,"[Crime,, Drama, A, jury, holdout, attempts, to..."


In [21]:
df1['tags'] = df1['tags'].apply(lambda x:" ".join(x))

In [22]:
df1['tags'] = df1['tags'].apply(lambda x:x.lower())

In [23]:
df1['tags'][0]

'drama two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency. frank darabont tim robbins morgan freeman bob gunton william sadler'

In [24]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000, stop_words ='english')

In [25]:
vectors = cv.fit_transform(df1['tags']).toarray()

In [26]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [27]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Stemming

In [28]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [29]:
def stem(text):
    y = []
    
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [30]:
df1['tags'] = df1['tags'].apply(stem)

Using cosine similarity for finding similar movies

In [31]:
from sklearn.metrics.pairwise import cosine_similarity

In [32]:
similarity = cosine_similarity(vectors)

In [33]:
def recommend(movie):
    movie_index = df1[df1['Series_Title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse = True, key = lambda x:x[1])[1:11]
    
    for i in movies_list:
        print(df1.iloc[i[0]].Series_Title)

In [34]:
sorted(list(enumerate(similarity[0])), reverse = True, key = lambda x:x[1])[1:11]

[(505, 0.21052631578947364),
 (167, 0.15974461276617435),
 (817, 0.1538967528127731),
 (197, 0.15018785229652765),
 (234, 0.14673479641335552),
 (697, 0.14673479641335552),
 (877, 0.14673479641335552),
 (311, 0.14350946197048195),
 (350, 0.14350946197048195),
 (958, 0.14350946197048195)]

**Final Output**

In [35]:
x = input('Please Enter the name of the movie:')
print('\nThe Top 10 recommendations are as follows:\n')
recommend(x)

Please Enter the name of the movie:The Dark Knight

The Top 10 recommendations are as follows:

Batman Begins
The Dark Knight Rises
The Prestige
Kill Bill: Vol. 1
Falling Down
3:10 to Yuma
Empire of the Sun
Joker
La battaglia di Algeri
The Deer Hunter


## 2. Sentiment Analysis of IMDB 50k movie reviews dataset

In [36]:
review = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Movie_Recommender_system_ and_sentiment_analysis/IMDB Dataset.csv')

In [37]:
review.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [38]:
review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


## Data preprocessing

**Checking for duplicate rows**

In [39]:
review.duplicated().sum()

418

In [40]:
review.drop_duplicates(inplace= True)

In [41]:
review.duplicated().sum()

0

**Considering only a sample of 5000 rows from the dataset for the analysis**

In [42]:
review = review.sample(5000)

In [43]:
review.head(5)

,review,sentiment
33888,This is clearly a French film. It is about you...,negative
28145,This is a rather tame fluff piece concerning W...,negative
19696,"Like most, I rented this after I heard the uni...",negative
15792,"I rented this flick for one reason Tom Savini,...",negative
20487,Oldboy is set in Korea & starts as a drunken D...,negative


In [44]:
review['sentiment'].replace({'positive':1, 'negative':0}, inplace=True)

**Removing HTML tags**

In [45]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

In [46]:
review['review']=review['review'].apply(strip_html)

**Removing Square brackets**

In [47]:
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

In [48]:
review['review']=review['review'].apply(remove_between_square_brackets)

**Converting text to lowercase**

In [49]:
def convert_lower(text):
  return text.lower()

In [50]:
review['review'] = review['review'].apply(convert_lower)

**Removing special characters**

In [51]:
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text

In [52]:
review['review'] = review['review'].apply(remove_special_characters)

In [53]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Removing Stopwords**

In [54]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [55]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [56]:
def remove_stopwords(text):
  new_text = []
  for word in text.split():
    if word not in stopwords.words('english'):
      new_text.append(word)
  y =new_text[:]

  new_text.clear()
  return y

In [57]:
review['review'] = review['review'].apply(remove_stopwords)

In [58]:
review

,review,sentiment
33888,"[clearly, french, film, young, group, idealist...",0
28145,"[rather, tame, fluff, piece, concerning, ww, i...",0
19696,"[like, rented, heard, universal, praise, despi...",0
15792,"[rented, flick, one, reason, tom, savini, resp...",0
20487,"[oldboy, set, korea, starts, drunken, daesu, o...",0
...,...,...
18779,"[figured, horror, film, orson, welles, would, ...",0
31581,"[king, kong, vs, godzilla, 1963, recall, rated...",0
14186,"[need, movie, plain, simple, original, althoug...",0
28189,"[brendan, filone, absolute, best, character, s...",1


In [59]:
def join_back(list_input):
  return " ".join(list_input)

In [60]:
review['review']=review['review'].apply(join_back)

**Stemming**

In [61]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [62]:
def simple_stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text= ' '.join([ps.stem(word) for word in text.split()])
    return text


In [63]:
review['review']=review['review'].apply(simple_stemmer)

In [64]:
review.head(5)

,review,sentiment
33888,clearli french film young group idealistrevolu...,0
28145,rather tame fluff piec concern ww ii code brok...,0
19696,like rent heard univers prais despit countless...,0
15792,rent flick one reason tom savini respect work ...,0
20487,oldboy set korea start drunken daesu oh minsik...,0


In [65]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

In [66]:
X = cv.fit_transform(review['review']).toarray()

In [67]:
X.shape

(5000, 40863)

In [68]:
y = review.iloc[:,-1].values

### Splitting the dataset into training and test set

In [69]:
from sklearn.model_selection import train_test_split

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.2)

Model fitting

In [71]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB 
from sklearn.svm import LinearSVC   
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report      

In [72]:
clf1 = GaussianNB()
clf2 = MultinomialNB()
clf3 = BernoulliNB()
clf4 = LinearSVC()

In [73]:
clf1.fit(X_train, y_train)
clf2.fit(X_train, y_train)
clf3.fit(X_train, y_train)
clf4.fit(X_train, y_train)

LinearSVC()

Model performance

In [74]:
y_pred1 = clf1.predict(X_test)
y_pred2 = clf2.predict(X_test)
y_pred3 = clf3.predict(X_test)
y_pred4 = clf4.predict(X_test)

In [75]:
print('Gaussian',accuracy_score(y_test, y_pred1))
print('Multinomial',accuracy_score(y_test, y_pred2))
print('Bernaulli',accuracy_score(y_test, y_pred3))
print('LinearSVC',accuracy_score(y_test, y_pred4))

Gaussian 0.628
Multinomial 0.835
Bernaulli 0.824
LinearSVC 0.83


In [76]:
print('CLASSIFICATION REPORTS\n\n')
print('1.Gaussian\n')
print(classification_report(y_test, y_pred1))
print('2.Multinomial\n')
print(classification_report(y_test, y_pred2))
print('3.Bernaulli\n')
print(classification_report(y_test, y_pred3))
print('4.LinearSVC\n')
print(classification_report(y_test, y_pred4))

CLASSIFICATION REPORTS


1.Gaussian

              precision    recall  f1-score   support

           0       0.62      0.70      0.66       505
           1       0.64      0.55      0.60       495

    accuracy                           0.63      1000
   macro avg       0.63      0.63      0.63      1000
weighted avg       0.63      0.63      0.63      1000

2.Multinomial

              precision    recall  f1-score   support

           0       0.83      0.84      0.84       505
           1       0.84      0.83      0.83       495

    accuracy                           0.83      1000
   macro avg       0.84      0.83      0.83      1000
weighted avg       0.84      0.83      0.83      1000

3.Bernaulli

              precision    recall  f1-score   support

           0       0.80      0.87      0.83       505
           1       0.85      0.78      0.81       495

    accuracy                           0.82      1000
   macro avg       0.83      0.82      0.82      1000
weighted 

So, we can see that with a dataset of 5000 reviews *Multinomial Naive Bayes* algorithim gives the highest accuracy of 83.5%. Bernailli Naive Bayes and LinearSVC algorithims also has accuracy values 82.4% and 83% respectively which is very close to multinomial Naive Bayes. 